# A/B Testing for ShoeFly.com

ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. 

This is an analysis of the data(using aggregate measures) to show the two ads are performing on each of the different platforms on each day of the week.

In [ ]:
import codecademylib3
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')

#How many views (i.e., rows of the table) came from each utm_source?
utm_count = ad_clicks.groupby('utm_source').day.count().reset_index()

#new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

#group by utm_source and is_click and count the number of user_id‘s in each of those groups. 
clicks_by_source = ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()

#pivot the data
clicks_pivot = clicks_by_source.pivot(columns='is_click',index='utm_source',values='user_id')

#new column in clicks_pivot called percent_clicked(percent of users who clicked on the ad from each utm_source)
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])  

#number of people shown both ads
ad_AB = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
ad_AB_clicks = ad_clicks.groupby(['experimental_group','is_click']).user_id.count().reset_index()

#pivot the data
ad_AB_clicks_pivot = ad_AB_clicks.pivot(columns='experimental_group', index='is_click',values='user_id')

#two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

#percent of users who clicked on the ad by day(For each group - a_clicks and b_clicks).
a_clicks['percent_clicked'] = (a_clicks.is_click == True)/ a_clicks.is_click.count() 
b_clicks['percent_clicked'] = (b_clicks.is_click == True)/ b_clicks.is_click.count() 

#percent clicked by the day of the week
A_clicks = a_clicks.groupby(['is_click','day']).percent_clicked.count().reset_index()
B_clicks = b_clicks.groupby(['is_click','day']).percent_clicked.count().reset_index()

#pivot the data
A_clicks_pivot = A_clicks.pivot(columns='day', index='is_click',values='percent_clicked')
B_clicks_pivot = B_clicks.pivot(columns='day', index='is_click',values='percent_clicked')

